In [1]:
import pandas as pd
import duckdb as db
import json
import os

In [2]:
# Load data
root_dir = './data/'
dist_center_df = pd.read_csv(os.path.join(root_dir,'distribution_centers.csv'))
order_items_df = pd.read_csv(os.path.join(root_dir,'order_items.csv'))
orders_df = pd.read_csv(os.path.join(root_dir,'orders.csv'))
products_df = pd.read_csv(os.path.join(root_dir,'products.csv'))
users_df = pd.read_csv(os.path.join(root_dir,'users.csv'))

## VALUE DUPLICATION CHECK

In [3]:
orders_df[orders_df['order_id'].duplicated()]

,order_id,user_id,status,gender,created_at,returned_at,shipped_at,delivered_at,num_of_item


In [4]:
orders_df.drop_duplicates(subset=['order_id'], keep='last', inplace=True)

In [5]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              100000 non-null  int64  
 1   first_name      100000 non-null  object 
 2   last_name       100000 non-null  object 
 3   email           100000 non-null  object 
 4   age             100000 non-null  int64  
 5   gender          100000 non-null  object 
 6   state           100000 non-null  object 
 7   street_address  100000 non-null  object 
 8   postal_code     100000 non-null  object 
 9   city            99069 non-null   object 
 10  country         100000 non-null  object 
 11  latitude        100000 non-null  float64
 12  longitude       100000 non-null  float64
 13  traffic_source  100000 non-null  object 
 14  created_at      100000 non-null  object 
dtypes: float64(2), int64(2), object(11)
memory usage: 11.4+ MB


In [6]:
users_df.head()

,id,first_name,last_name,email,age,gender,state,street_address,postal_code,city,country,latitude,longitude,traffic_source,created_at
0,40112,Robert,Martin,robertmartin@example.com,41,M,Mie,8560 Paul Station Apt. 589,513-0836,Suzuka City,Japan,34.851814,136.508713,Search,2019-03-01 01:22:00 UTC
1,4646,Stephen,Dudley,stephendudley@example.net,60,M,Acre,6531 Dixon Trace,69917-400,Rio Branco,Brasil,-9.945568,-67.835610,Search,2023-02-14 18:56:00 UTC
2,5801,David,Smith,davidsmith@example.com,38,M,Acre,403 Chris Island,69917-400,Rio Branco,Brasil,-9.945568,-67.835610,Search,2023-08-13 00:55:22.961196 UTC
3,22001,Timothy,Anderson,timothyanderson@example.org,14,M,Acre,494 Anna Unions Suite 031,69917-400,Rio Branco,Brasil,-9.945568,-67.835610,Search,2022-12-03 11:15:00 UTC
4,39207,Erik,Johnson,erikjohnson@example.net,21,M,Acre,01619 Jermaine Summit,69917-400,Rio Branco,Brasil,-9.945568,-67.835610,Search,2021-08-21 18:41:00 UTC


In [7]:
def query(sql_query):
    return db.sql(sql_query).df()

In [8]:
null_value = query(
    "SELECT * FROM users_df WHERE city is null"
)
null_value.head()

,id,first_name,last_name,email,age,gender,state,street_address,postal_code,city,country,latitude,longitude,traffic_source,created_at
0,9303,Timothy,Edwards,timothyedwards@example.org,51,M,Acre,694 Kyle View,69980-000,None,Brasil,-8.065346,-72.870949,Facebook,2019-08-24 17:55:00 UTC
1,25691,Whitney,Acosta,whitneyacosta@example.net,26,F,Acre,6221 Ethan Stravenue,69980-000,None,Brasil,-8.065346,-72.870949,Search,2020-12-28 01:27:00 UTC
2,35279,Gina,Valenzuela,ginavalenzuela@example.com,21,F,Acre,301 Jillian Highway Apt. 557,69980-000,None,Brasil,-8.065346,-72.870949,Search,2022-12-07 08:44:00 UTC
3,49234,Michelle,Carter,michellecarter@example.com,58,F,Acre,50990 Laurie Plaza,69980-000,None,Brasil,-8.065346,-72.870949,Display,2022-07-12 14:02:00 UTC
4,56982,Benjamin,Ellison,benjaminellison@example.net,19,M,Acre,762 Christopher Mountain Suite 437,69980-000,None,Brasil,-8.065346,-72.870949,Search,2023-08-12 18:18:56.612255 UTC


In [9]:
null_total = query(
    "SELECT latitude, longitude, COUNT(id) as null_total FROM null_value GROUP BY latitude, longitude"
)
null_total

,latitude,longitude,null_total
0,-12.420754,-38.943214,5
1,-1.552182,-47.612929,7
2,-6.771137,-39.290858,13
3,-9.789232,-36.849023,12
4,28.587128,-81.183958,4
...,...,...,...
132,-20.372044,-40.376547,6
133,-15.850637,-48.064887,10
134,35.223806,128.587605,2
135,40.317547,-3.548108,2


In [10]:
!pip install geopy


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
from geopy.geocoders import Nominatim

In [12]:
null_total.loc[17].to_dict()

{'latitude': 36.66293851, 'longitude': -4.511719984, 'null_total': 2.0}

In [13]:
coor = null_total['latitude'][14], null_total['longitude'][14]
coordinate = f"{coor[0]}, {coor[1]}"
coordinate

'37.33709112, -76.7569565'

In [14]:
geolocator = Nominatim(user_agent="Askar")
location = geolocator.reverse(coordinate)
location.raw

{'place_id': 2765162,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 368370843,
 'lat': '37.33644285',
 'lon': '-76.75767138990827',
 'class': 'amenity',
 'type': 'parking',
 'place_rank': 30,
 'importance': 9.99999999995449e-06,
 'addresstype': 'amenity',
 'name': '',
 'display_name': 'Opportunity Way, Marstons Mobile Home Park, Lightfoot, James City County, Virginia, 23090, United States',
 'address': {'road': 'Opportunity Way',
  'hamlet': 'Marstons Mobile Home Park',
  'village': 'Lightfoot',
  'county': 'James City County',
  'state': 'Virginia',
  'ISO3166-2-lvl4': 'US-VA',
  'postcode': '23090',
  'country': 'United States',
  'country_code': 'us'},
 'boundingbox': ['37.3359150', '37.3369451', '-76.7585350', '-76.7569739']}

In [15]:
def search_city(latitude, longitude):
    geolocate = Nominatim(user_agent="Askar")
    coordinates = f"{latitude},{longitude}"
    location = geolocate.reverse(coordinates)
    return location.raw.get('address',{}).get('city')

In [16]:
test = print(search_city(51.76537677, 8.54063881))

None


In [18]:
null_city = list(null_total[['latitude','longitude']].values)
count = 1
city_code = {}
for location in null_city[::-1]:
    location_key = f'{location[0]}/{location[1]}'
    if location_key not in city_code:
        city = search_city(location[0],location[1])
        city_code[location_key] = city
    print(f'Detecting location {count} from {location} --> {city_code[location_key]}')
    count+=1

Detecting location 1 from [40.26785722 -3.77774404] --> Fuenlabrada
Detecting location 2 from [40.31754681 -3.54810806] --> None
Detecting location 3 from [ 35.22380553 128.5876055 ] --> 창원시
Detecting location 4 from [-15.85063718 -48.06488677] --> Taguatinga
Detecting location 5 from [-20.37204429 -40.37654705] --> Cariacica
Detecting location 6 from [ 36.35354657 -78.38609351] --> None
Detecting location 7 from [ -7.45008935 -35.13419726] --> None
Detecting location 8 from [ -3.295962   -43.33373854] --> None
Detecting location 9 from [ -3.28907196 -46.04148814] --> None
Detecting location 10 from [41.6208308   0.65841549] --> Lleida
Detecting location 11 from [ 37.12583037 -76.44706339] --> None
Detecting location 12 from [ 37.47429329 -77.58517125] --> None
Detecting location 13 from [ 38.21639996 -77.42933948] --> None
Detecting location 14 from [ 38.40333075 -77.55210011] --> None
Detecting location 15 from [ 38.6539842  -77.42752508] --> None
Detecting location 16 from [ -1.3723

In [19]:
import json

with open("city_code.json", "w") as outfile:
    json.dump(city_code, outfile, indent=4)

In [20]:
users_df['filled_data'] = None
users_df.head()

,id,first_name,last_name,email,age,gender,state,street_address,postal_code,city,country,latitude,longitude,traffic_source,created_at,filled_data
0,40112,Robert,Martin,robertmartin@example.com,41,M,Mie,8560 Paul Station Apt. 589,513-0836,Suzuka City,Japan,34.851814,136.508713,Search,2019-03-01 01:22:00 UTC,None
1,4646,Stephen,Dudley,stephendudley@example.net,60,M,Acre,6531 Dixon Trace,69917-400,Rio Branco,Brasil,-9.945568,-67.835610,Search,2023-02-14 18:56:00 UTC,None
2,5801,David,Smith,davidsmith@example.com,38,M,Acre,403 Chris Island,69917-400,Rio Branco,Brasil,-9.945568,-67.835610,Search,2023-08-13 00:55:22.961196 UTC,None
3,22001,Timothy,Anderson,timothyanderson@example.org,14,M,Acre,494 Anna Unions Suite 031,69917-400,Rio Branco,Brasil,-9.945568,-67.835610,Search,2022-12-03 11:15:00 UTC,None
4,39207,Erik,Johnson,erikjohnson@example.net,21,M,Acre,01619 Jermaine Summit,69917-400,Rio Branco,Brasil,-9.945568,-67.835610,Search,2021-08-21 18:41:00 UTC,None


In [21]:
filled_city = query(
    "SELECT distinct postal_code, country, city, latitude, longitude, filled_data FROM users_df WHERE city is null"
)
filled_city

,postal_code,country,city,latitude,longitude,filled_data
0,68230-000,Brasil,None,0.284273,-53.893255,NaN
1,68250-000,Brasil,None,-0.171398,-55.688365,NaN
2,68330-000,Brasil,None,-2.184619,-52.547676,NaN
3,68540-000,Brasil,None,-8.182462,-49.512580,NaN
4,69600-000,Brasil,None,-4.496417,-69.449963,NaN
...,...,...,...,...,...,...
132,631-820,South Korea,None,35.149606,128.460248,NaN
133,28524,Spain,None,40.317547,-3.548108,NaN
134,28909,Spain,None,40.300637,-3.630132,NaN
135,28946,Spain,None,40.267857,-3.777744,NaN


In [22]:
with open("city_code.json", "r") as fill_city:
    city_code = json.load(fill_city)
city_code

{'40.26785722/-3.77774404': 'Fuenlabrada',
 '40.31754681/-3.548108063': None,
 '35.22380553/128.5876055': '창원시',
 '-15.85063718/-48.06488677': 'Taguatinga',
 '-20.37204429/-40.37654705': 'Cariacica',
 '36.35354657/-78.38609351': None,
 '-7.450089353/-35.13419726': None,
 '-3.295961998/-43.33373854': None,
 '-3.28907196/-46.04148814': None,
 '41.6208308/0.6584154877': 'Lleida',
 '37.12583037/-76.44706339': None,
 '37.47429329/-77.58517125': None,
 '38.21639996/-77.42933948': None,
 '38.40333075/-77.55210011': None,
 '38.6539842/-77.42752508': None,
 '-1.372387836/-59.98896544': None,
 '-4.496416746/-69.44996292': None,
 '-3.779835447/-60.69798501': None,
 '-6.458133124/-61.4460949': None,
 '-3.770435805/-60.18321229': None,
 '-3.46271025/-40.1673669': None,
 '1.096832853/-53.30566415': None,
 '30.0397354/-95.53316186': None,
 '-0.2638776812/-50.72755912': None,
 '-1.981978652/-46.43637028': None,
 '-1.15180429/-51.55505639': None,
 '-11.19130533/-38.20592756': None,
 '-10.00872906/-43.1

In [23]:
users_df['filled_data'] = users_df.apply(
    lambda x: city_code.get(f"{x['latitude']}/{x['longitude']}", None) if pd.isnull(x["city"])
    else (x["city"]), axis=1
)

In [24]:
new_data = query(
    "SELECT distinct postal_code, city, country, latitude, longitude, filled_data FROM users_df WHERE city is null"
)
new_data

,postal_code,city,country,latitude,longitude,filled_data
0,68230-000,None,Brasil,0.284273,-53.893255,None
1,68330-000,None,Brasil,-2.184619,-52.547676,None
2,68540-000,None,Brasil,-8.182462,-49.512580,None
3,68695-000,None,Brasil,-2.910334,-48.736935,None
4,69190-000,None,Brasil,-4.983694,-58.060068,None
...,...,...,...,...,...,...
132,29142,None,Brasil,-20.372044,-40.376547,Cariacica
133,630-490,None,South Korea,35.223806,128.587605,창원시
134,630-850,None,South Korea,35.231193,128.513836,창원시
135,30835,None,Spain,37.966542,-1.297636,Murcia


In [25]:
query(
    "SELECT * FROM products_df WHERE name is null"
)

,id,cost,category,name,brand,retail_price,department,sku,distribution_center_id
0,24455,67.335453,Outerwear & Coats,None,Tru-Spec,147.990005,Men,B290A635641F585B3DD6B95FD42DC267,2
1,12586,18.972000,Intimates,None,Josie by Natori,36.000000,Women,A7EA034186E14FB5F7B37CF664893CD2,1


In [26]:
products_df[products_df.isnull().any(axis=1)]

,id,cost,category,name,brand,retail_price,department,sku,distribution_center_id
173,27543,19.872000,Swim,Harbor Bay Big & Tall Swim Rash Guard,NaN,36.000000,Men,50E9EA9C1BC5E6067AB7C0D8FBDD5F21,3
174,27640,29.353500,Swim,O'Neill Hangover Boardshorts,NaN,49.500000,Men,42ADA6A116005062613A49ADD8AB908B,3
1085,21207,30.754350,Jeans,Ariat 9424 Men's M4 Low Rise Boot Tumbled,NaN,59.950001,Men,A59687B426E9EB30465997EB54862A2E,3
1086,21484,30.285500,Jeans,True Nation Big & Tall Relaxed-Fit Jeans,NaN,59.500000,Men,64C32B8EB5304D80BA093227975ECE13,3
3339,25135,12.476400,Socks,Volcom Men's Argyle Socks,NaN,22.480000,Men,0DC76D68D436BB030EFBE3A8386EE47A,3
3340,25187,14.425000,Socks,SockGuy Men's Kokopelli Socks,NaN,25.000000,Men,43A4D135B5225238F2ADBF88DC810A6B,3
8837,16309,11.520000,Tops & Tees,Hurley Men's One and Only Short Sleeve Tee,NaN,20.000000,Men,88E0F16114A1E011C87B797513095A20,3
8838,16559,14.944250,Tops & Tees,Gildan Adult Ultra CottonTM Sleeveless T-Shirt,NaN,25.990000,Men,DC0FC2D90D6102BA02D0206857EAAC51,3
8839,16898,13.925000,Tops & Tees,Quiksilver Waterman Men's On The Rise,NaN,25.000000,Men,22811EE19846217512507785E74D12CC,3
9961,27288,12.726610,Sleep & Lounge,White and Black Check Dot VIP Flannel Pants fo...,NaN,28.990000,Men,97FB74BBDD02677ADC1A871FDD16AC7A,3


In [27]:
query(
    "SELECT * FROM products_df WHERE name is null"
)

,id,cost,category,name,brand,retail_price,department,sku,distribution_center_id
0,24455,67.335453,Outerwear & Coats,None,Tru-Spec,147.990005,Men,B290A635641F585B3DD6B95FD42DC267,2
1,12586,18.972000,Intimates,None,Josie by Natori,36.000000,Women,A7EA034186E14FB5F7B37CF664893CD2,1


In [28]:
products_df['name'] = products_df['name'].fillna('Unknown')
products_df[products_df['id'].isin([24455,12586])]

,id,cost,category,name,brand,retail_price,department,sku,distribution_center_id
10713,24455,67.335453,Outerwear & Coats,Unknown,Tru-Spec,147.990005,Men,B290A635641F585B3DD6B95FD42DC267,2
20171,12586,18.972000,Intimates,Unknown,Josie by Natori,36.000000,Women,A7EA034186E14FB5F7B37CF664893CD2,1


In [29]:
products_df['retail_price'].min()

0.0199999995529651

In [30]:
mode = products_df['retail_price'].mode()

In [31]:
products_df['retail_price'] = products_df['retail_price'].fillna(mode)

## VALUE VALIDITY CHECK

In [32]:
products_df.query("retail_price <= 0")

,id,cost,category,name,brand,retail_price,department,sku,distribution_center_id


In [33]:
order_items_df.query("sale_price <= 0")

,id,order_id,user_id,product_id,inventory_item_id,status,created_at,shipped_at,delivered_at,returned_at,sale_price
